# Building powerful image classification models using very little data

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

Using TensorFlow backend.


In [2]:
img = load_img('data/train/cats/cat.0.jpg') # this is a PIL image
x = img_to_array(img) # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape) # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to t he 'preview/' directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='preview', save_prefix='cat',
                         save_format='jpeg'):
    i += 1
    if i > 20:
        break # otherwise the generator would loop indefinitely

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten()) # this converts our 3D feature maps to 1D features vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop',
             metrics=['accuracy'])

batch_size=16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolders of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
    'data/train', # this is the target directory
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary') # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
    'data/validation',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=800 // batch_size)
model.save_weights('first_try.h5') # always save your weights after training or during training

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 14s - loss: 0.7131 - acc: 0.5000 - val_loss: 0.6924 - val_acc: 0.5000
Epoch 2/50
125/125 [==============================] - 13s - loss: 0.6907 - acc: 0.5760 - val_loss: 0.6465 - val_acc: 0.6450
Epoch 3/50
125/125 [==============================] - 13s - loss: 0.6313 - acc: 0.6365 - val_loss: 0.6266 - val_acc: 0.6275
Epoch 4/50
125/125 [==============================] - 13s - loss: 0.6001 - acc: 0.6715 - val_loss: 0.5923 - val_acc: 0.6737
Epoch 5/50
125/125 [==============================] - 13s - loss: 0.5819 - acc: 0.7070 - val_loss: 0.5972 - val_acc: 0.6987- los - ETA: 0s - loss: 0.5837 - acc: 0.70
Epoch 6/50
125/125 [==============================] - 13s - loss: 0.5665 - acc: 0.7160 - val_loss: 0.5794 - val_acc: 0.6700
Epoch 7/50
125/125 [==============================] - 13s - loss: 0.5482 - acc: 0.7325 - val_loss: 0.5560 - val_acc: 0.7262
Epoch 8

In [4]:
from keras import applications
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

batch_size=16

dataget = ImageDataGenerator(rescale=1./255)
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')
generator = dataget.flow_from_directory('data/train',
                                       target_size=(150,150),
                                       batch_size=batch_size,
                                       class_mode=None, # this means our generator will yield only batches of data, no labels
                                       shuffle=False) # our data will be in order, so all first 1000 images will be cats, then 1000 dogs-cats.ipynb
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, (2000 // batch_size))

# save the output as numpy arrays
#np.save(open('bottleneck_features_train.npy', 'w'), np.array2string(bottleneck_features_train, precision=8))
with open('bottleneck_features_train.npy', 'wb') as features_train_file:
    np.save(features_train_file, bottleneck_features_train)

generator = datagen.flow_from_directory('data/validation',
                                       target_size=(150,150),
                                       batch_size=batch_size,
                                       class_mode=None,
                                       shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, (800 // batch_size))
#np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)
with open('bottleneck_features_validation.npy', 'wb') as features_validation_file:
    np.save(features_validation_file, bottleneck_features_validation)



Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [5]:
with open('bottleneck_features_train.npy', 'rb') as features_train:
    train_data = np.load(features_train)

# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

with open('bottleneck_features_validation.npy', 'rb') as features_validation:
    validation_data = np.load(features_validation)
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          
          epochs=50, verbose=2,
          batch_size=batch_size,
          #steps_per_epoch=(train_data.shape[0] // batch_size),
          validation_data=(validation_data, validation_labels))
          #validation_steps=(validation_data.shape[0] // batch_size))

model.save_weights('bottleneck_fc_model.h5')


Train on 2000 samples, validate on 800 samples
Epoch 1/50
0s - loss: 0.6468 - acc: 0.7580 - val_loss: 3.0678 - val_acc: 0.7837
Epoch 2/50
0s - loss: 0.3541 - acc: 0.8490 - val_loss: 3.3112 - val_acc: 0.7812
Epoch 3/50
0s - loss: 0.3055 - acc: 0.8845 - val_loss: 4.5587 - val_acc: 0.7025
Epoch 4/50
0s - loss: 0.2896 - acc: 0.8880 - val_loss: 3.0158 - val_acc: 0.8063
Epoch 5/50
0s - loss: 0.2272 - acc: 0.9115 - val_loss: 2.8105 - val_acc: 0.8150
Epoch 6/50
0s - loss: 0.1954 - acc: 0.9310 - val_loss: 3.7945 - val_acc: 0.7550
Epoch 7/50
0s - loss: 0.1825 - acc: 0.9325 - val_loss: 3.2694 - val_acc: 0.7900
Epoch 8/50
0s - loss: 0.1622 - acc: 0.9380 - val_loss: 3.2554 - val_acc: 0.7925
Epoch 9/50
0s - loss: 0.1513 - acc: 0.9460 - val_loss: 3.3945 - val_acc: 0.7788
Epoch 10/50
0s - loss: 0.1189 - acc: 0.9545 - val_loss: 2.9388 - val_acc: 0.8137
Epoch 11/50
0s - loss: 0.1377 - acc: 0.9580 - val_loss: 3.1308 - val_acc: 0.8013
Epoch 12/50
0s - loss: 0.0924 - acc: 0.9685 - val_loss: 2.7782 - val_ac

In [6]:
# build the VGG16 network
vgg16model = applications.VGG16(weights='imagenet', include_top=False,
                               input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tunning
top_model.load_weights('bottleneck_fc_model.h5')

# add the model on top of the convolutional base
#vgg16model.add(top_model)
# since the line above crashes with "'Model' object has not attribute 'add'"
# we try the following instead
model = Model(inputs=vgg16model.input, outputs=top_model(vgg16model.output))

Model loaded.


In [8]:
from keras import optimizers
# set the first 25 layers (up to t he last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
             metrics=['accuracy'])

In [12]:
batch_size = 16

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'data/validation',
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=2000//batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=800//batch_size, verbose=2)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
27s - loss: 0.8670 - acc: 0.8965 - val_loss: 1.0482 - val_acc: 0.8875
Epoch 2/50
27s - loss: 0.9299 - acc: 0.8905 - val_loss: 1.0245 - val_acc: 0.8938
Epoch 3/50
27s - loss: 0.9257 - acc: 0.8805 - val_loss: 1.0246 - val_acc: 0.8850
Epoch 4/50
27s - loss: 0.9433 - acc: 0.8865 - val_loss: 1.0837 - val_acc: 0.8862
Epoch 5/50
27s - loss: 0.8761 - acc: 0.8880 - val_loss: 1.0127 - val_acc: 0.8925
Epoch 6/50
27s - loss: 0.7607 - acc: 0.9030 - val_loss: 1.0482 - val_acc: 0.8875
Epoch 7/50
27s - loss: 0.8917 - acc: 0.8875 - val_loss: 1.0476 - val_acc: 0.8850
Epoch 8/50
27s - loss: 0.8720 - acc: 0.8955 - val_loss: 1.0306 - val_acc: 0.8938
Epoch 9/50
27s - loss: 0.9738 - acc: 0.8890 - val_loss: 1.0960 - val_acc: 0.8825
Epoch 10/50
27s - loss: 0.9056 - acc: 0.8830 - val_loss: 1.0664 - val_acc: 0.8875
Epoch 11/50
27s - loss: 0.7629 - acc: 0.9015 - val_loss: 1.0482 - val_acc: 0.8875
Epoch 12/50
27s - loss: 

Here are a few more approaches you can try to get to above 0.95:
    * more aggresive data augmentation
    * more aggresive dropout
    * use of L1 and L2 regularization (also known as "weight decay")
    * fine-tuning one more convolutional block (alongside greater regularization)
 
 Here is where you can fine the code for our examples:
 
     * Convnet trained from scratch:
     https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d
     
     * Bottleneck features
     https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069
     
     *Fine-tuning
     https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975